In [7]:
import pandas as pd
import numpy as np 
import requests
import ssl
from bs4 import BeautifulSoup 

In [9]:
url = "https://predoc.org/opportunities"
response = requests.get(url, verify=False)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
else:
    print("Error: Unable to fetch the URL")

/Users/annaking/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'predoc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [131]:
import re

# Find all the article elements containing predoctoral positions
positions = []

for article in articles:
    try:
        title_element = article.find("h2")
        title = title_element.text.strip()
        link = title_element.find("a")["href"]

        details_element = article.find("p")

        # Split the details by line breaks (<br>) using a regular expression
        details_lines = [line.strip() for line in re.split('<br ?/?>', details_element.decode_contents())]

        sponsoring_researchers = details_lines[0].replace("Sponsoring Researchers:", "").replace("Sponsoring Researcher:", "").strip()
        institution = details_lines[1].replace("Institution:", "").strip()
        fields_of_research = details_lines[2].replace("Fields of Research:", "").strip()

        # Combine any remaining lines as the deadline
        deadline = " ".join(details_lines[3:]).replace("Deadline:", "").strip()

        position = {
            "name": title,
            "link": link,
            "sponsoring_researchers": sponsoring_researchers,
            "institution": institution,
            "fields_of_research": fields_of_research,
            "deadline": deadline,
        }
        positions.append(position)
    except IndexError:
        print(f"Warning: Failed to parse position '{title}' due to incomplete or unexpected data")

df = pd.DataFrame(positions)


In [155]:
#Clean dataframe
df_copy.institution = df_copy.institution.str.replace('<em>',"")
df_copy.institution = df_copy.institution.str.replace('</em>',"")
df_copy.fields_of_research = df_copy.fields_of_research.str.replace('Fields of research: ',"")
df_copy.fields_of_research = df_copy.fields_of_research.str.replace('Field of research: ',"")
df_copy.fields_of_research = df_copy.fields_of_research.str.replace('</span>',"")

df_copy.fields_of_research = df_copy.fields_of_research.str.replace('<span style="background-color: #ffffff; text-align: start; letter-spacing: normal; color: #000000;">',"")
df_copy.fields_of_research = df_copy.fields_of_research.str.replace('<span style="color: #242424; background-color: #ffffff;">',"")
df_copy.fields_of_research = df_copy.fields_of_research.str.replace('<span class="x_Apple-converted-space" style="color: #242424; background-color: #ffffff; margin: 0px; padding: 0px; border: 0px;">',"")
df_copy.fields_of_research = df_copy.fields_of_research.str.replace('<span style="background-color: #ffffff; color: #000000;">',"")
df_copy.fields_of_research = df_copy.fields_of_research.str.replace('</em>',"")

#lookup where df contains 
    #printing = df_copy[df_copy.fields_of_research.str.contains('span')]
    #printing.fields_of_research.iloc[1]


In [156]:
df.to_csv('predoc.csv')
